<a href="https://colab.research.google.com/github/genterist/BKT-Jupyter/blob/master/BKT_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://brand.ncsu.edu/img/logo/brick2x2.jpg =170x)

## **DEPARTMENT OF COMPUTER SCIENCE - CSC591 FALL 2018 Edu Tech**
	
###TAM N. NGUYEN

In [2]:
# PLEASE RUN THIS FIRST

import sys
version = sys.version_info[0], sys.version_info[1]
print("Running on Python {}.{}".format(version[0], version[1]))

# Imports for all questions
import math as m
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt
#from sklearn import preprocessing, svm
#from sklearn.linear_model import LinearRegression
#from sklearn.model_selection import LeaveOneOut
#from sklearn import metrics
#from math import sqrt
#from keras.models import Sequential
#from keras.layers import Dense
#from itertools import combinations
#
# ATTENTION
#
# Please read README file to run codes properly in Jupyter Notebook
#
# Running on Google Colab notebook will require manual upload of CSV files to the "Files" section
# in the left column of the interface. 


Running on Python 2.7


### DATA IMPORT and PRE-PROCESSING

In [4]:
# for Google Colab, make sure you upload data at the beginning of EACH new session
# make sure you run code block 1 of this jupyter notebook (for imports)
#
raw_data = pd.read_csv('./data.csv', sep=',')
raw_data.shape

(36627, 10)

In [6]:
# original structure
raw_data.head(3)

,Student,StepID,Correct,KC_1,KC_27,KC_24,KC_14,KC_22,KC_20,KC_21
0,stu1,478,0,0,1,0,0,0,0,0
1,stu1,479,1,0,1,0,0,0,0,0
2,stu1,484,1,0,1,0,0,0,0,0


In [19]:
# turning it to a multi-index DF
mod1_data = raw_data.set_index(['Student','StepID'])

#make sure everything is in order
mod1_data.sortlevel(inplace=True)

# insert collumns for the probabilities
# hard-coding of columns is fine here since this is a simulated scenario
# real life situations may involve json or
# regular db which is much more organized

mod1_data.insert(2,'P(L)1',0)
mod1_data.insert(3,'P(C)1',0)

mod1_data.insert(5,'P(L)27',0)
mod1_data.insert(6,'P(C)27',0)

mod1_data.insert(8,'P(L)24',0)
mod1_data.insert(9,'P(C)24',0)

mod1_data.insert(11,'P(L)14',0)
mod1_data.insert(12,'P(C)14',0)

mod1_data.insert(14,'P(L)22',0)
mod1_data.insert(15,'P(C)22',0)

mod1_data.insert(17,'P(L)20',0)
mod1_data.insert(18,'P(C)20',0)

mod1_data.insert(20,'P(L)21',0)
mod1_data.insert(21,'P(C)21',0)

# only P(C) matter since it has other Probabilities embedded in it

mod1_data.head(5)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: sortlevel is deprecated, use sort_index(level= ...)
  after removing the cwd from sys.path.


Correct  KC_1  P(L)1  P(C)1  KC_27  P(L)27  P(C)27  KC_24  \
Student StepID                                                              
stu1    478           0     0      0      0      1       0       0      0   
        479           1     0      0      0      1       0       0      0   
        484           1     0      0      0      1       0       0      0   
        485           1     0      0      0      1       0       0      0   
        497           1     0      0      0      0       0       0      0   

                P(L)24  P(C)24   ...    P(C)14  KC_22  P(L)22  P(C)22  KC_20  \
Student StepID                   ...                                           
stu1    478          0       0   ...         0      0       0       0      0   
        479          0       0   ...         0      0       0       0      0   
        484          0       0   ...         0      0       0       0      0   
        485          0       0   ...         0      0       0       0      0   
        497          0       0   ...         0      1       0       0      0   

                P(L)20  P(C)20  KC_21  P(L)21  P(C)21  
Student StepID                                         
stu1    478          0       0      0       0       0  
        479          0       0      0       0       0  
        484          0       0      0       0       0  
        485          0       0      0       0       0  
        497          0       0      0       0       0  

[5 rows x 22 columns]

In [13]:

mod1_data.head(5)

Correct  KC_1  P  KC_27  KC_24  KC_14  KC_22  KC_20  KC_21
Student StepID                                                            
stu1    478           0     0  0      1      0      0      0      0      0
        479           1     0  0      1      0      0      0      0      0
        484           1     0  0      1      0      0      0      0      0
        485           1     0  0      1      0      0      0      0      0
        497           1     0  0      0      0      0      1      0      0